# Creación de Base de Datos Postgres a partir de licitaciones públicas (XML)

Este notebook muestra paso a paso cómo:
1. **Leer** un archivo XML con información de licitaciones públicas.
2. **Descargar y parsear** diferentes tablas de códigos desde URLs oficiales (contratación del estado).
3. **Transformar** la información en distintos DataFrames con `pandas`.
4. **Crear** tablas en una base de datos PostgreSQL usando `SQLAlchemy`.
5. **Insertar** los datos en dichas tablas.


## 1. Instalación de paquetes necesarios

En esta sección instalamos (en caso de que fuera necesario) los paquetes de Python que usaremos:
- `pandas` para la manipulación de datos.
- `lxml` para parsear XML.
- `requests` para hacer peticiones HTTP (descarga de ficheros y datos).
- `psycopg2-binary` y `sqlalchemy` para conectarnos y trabajar con PostgreSQL.

En tu entorno, si no los tuvieras instalados, podrías ejecutar:
```bash
!pip install pandas lxml requests psycopg2-binary sqlalchemy
```
O usar la magia `%pip` en Jupyter:
```bash
%%capture --no-stderr
%pip install --upgrade --quiet pandas lxml requests psycopg2-binary sqlalchemy
```

In [ ]:
import pandas as pd
import requests
from lxml import etree
from io import BytesIO

print("Paquetes importados correctamente.")

## 2. Lectura del documento XML principal

En esta sección se abre el documento XML local que contiene la información de licitaciones.
 - `file_path`: ruta donde se ubica el archivo `.atom` (o `.xml`).
 - Se utiliza `BytesIO` porque `lxml` trabaja con flujos binarios.
 - `ns_clean=True` ayuda a limpiar definiciones de espacio de nombres duplicadas o irrelevantes.


In [ ]:
# Carga del archivo XML local
file_path = r"C:\Users\Rodrigo\Documents\TFM\Documentos\licitacionesPerfilesContratanteCompleto3_202401\licitacionesPerfilesContratanteCompleto3.atom"

with open(file_path, 'rb') as file:
    xml_content = file.read()

parser = etree.XMLParser(ns_clean=True)
tree = etree.parse(BytesIO(xml_content), parser)
root = tree.getroot()

print("XML cargado y parseado correctamente.")

## 3. Descarga y parseo de tablas de códigos

En esta sección se definen las URLs de distintos códigos (p. ej. tipos de contrato, países, etc.),
luego se descargan y se parsean para transformarlos en DataFrames:

- `contract_codes`
- `service_contract_codes`
- `goods_contract_codes`
- `patrimonial_contract_codes`
- `country_codes`
- `country_subentity_codes`

Se utiliza la convención `ns_tables` para hacer referencia al espacio de nombres de `genericode` de OASIS.


In [ ]:
# Diccionario con los enlaces a las tablas de códigos
links_dict = {
    "contract_codes": "https://contrataciondelestado.es/codice/cl/2.08/ContractCode-2.08.gc",
    "service_contract_codes": "http://contrataciondelestado.es/codice/cl/1.04/ServiceContractCode-1.04.gc",
    "goods_contract_codes": "https://contrataciondelestado.es/codice/cl/1.04/GoodsContractCode-1.04.gc",
    "patrimonial_contract_codes": "https://contrataciondelestado.es/codice/cl/2.02/PatrimonialContractCode-2.02.gc",
    "country_codes": "http://contrataciondelestado.es/codice/cl/2.08/CountryIdentificationCode-2.08.gc",
    "country_subentity_codes": "http://contrataciondelestado.es/codice/cl/2.08/NUTS-2021.gc"
}

# Diccionario para almacenar un DataFrame por cada clave
dataframes_dict = {}

# Espacio de nombres para OASIS Genericode
ns_tables = {'gc': 'http://docs.oasis-open.org/codelist/ns/genericode/1.0/'}

for key, url in links_dict.items():
    print(f"Descargando y parseando: {key} -> {url}")
    
    # Descargamos el contenido del XML de códigos
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error al descargar {url} (status code: {response.status_code})")
        # Si no se pudo descargar, creamos un DataFrame vacío
        dataframes_dict[key] = pd.DataFrame()
        continue
    
    xml_content = response.content
    
    # Parseamos el contenido con lxml
    table_root = etree.XML(xml_content)
    
    # Extraemos las filas usando local-name() para las etiquetas
    rows = table_root.xpath('//gc:CodeList//*[local-name()="SimpleCodeList"]//*[local-name()="Row"]', namespaces=ns_tables)
    
    data = []
    for row in rows:
        # Extraemos el código
        code_list = row.xpath('./*[local-name()="Value"][@ColumnRef="code"]/*[local-name()="SimpleValue"]/text()')
        # Extraemos el nombre
        nombre_list = row.xpath('./*[local-name()="Value"][@ColumnRef="nombre"]/*[local-name()="SimpleValue"]/text()')

        code_val = code_list[0] if code_list else None
        nombre_val = nombre_list[0] if nombre_list else None

        data.append({
            key[:-1]: code_val,  # Ej: 'contract_codes' -> 'contract_code'
            key[:-6]+"_name": nombre_val  # Ej: 'contract_codes' -> 'contract_name'
        })
    
    # Creamos el DataFrame para la clave actual
    df = pd.DataFrame(data)
    dataframes_dict[key] = df

# Nombramos DataFrames finales para usarlos más fácilmente
df_contract_codes = dataframes_dict["contract_codes"]
df_service_contract_codes = dataframes_dict["service_contract_codes"]
df_goods_contract_codes = dataframes_dict["goods_contract_codes"]
df_patrimonial_contract_codes = dataframes_dict["patrimonial_contract_codes"]
df_country_codes = dataframes_dict["country_codes"]
df_country_subentity_codes = dataframes_dict["country_subentity_codes"]

# Creamos columna con el country_code a partir de las dos primeras letras de la subentidad
df_country_subentity_codes["country_code"] = df_country_subentity_codes["country_subentity_code"].str[:2]

# Eliminamos las filas de subentidades que no tengan país asociado
df_country_subentity_codes = df_country_subentity_codes[
    df_country_subentity_codes['country_code'].isin(df_country_codes['country_code'])
]

print("\nDataFrames creados correctamente.")

## 4. Parseo de las entradas del XML principal

Se definen espacios de nombres específicos para la estructura del documento XML que contiene la información de las licitaciones. Luego se crea un bucle que recorre cada `entry` y extrae los campos relevantes, como:

- `entry_id`, `link`, `summary`, `title`, `updated`
- `contract_folder_id`, `contract_folder_status`
- Tipo de proyecto, Subtipo (usando la correspondencia con los DataFrames de códigos)
- Montos económicos
- Información de la entidad contratante, guardándose en un DataFrame separado (`parties_df`).
- Documentos asociados (pliegos, anexos, etc.) en `documents_df`.

También se hace un mapeo de códigos de estado (`status_codes_dict`) para mostrar el texto en lugar del código.


In [ ]:
# Definición de los espacios de nombres para el XML de licitaciones
ns = {
    'atom': 'http://www.w3.org/2005/Atom',
    'cac': 'urn:dgpe:names:draft:codice:schema:xsd:CommonAggregateComponents-2',
    'cbc': 'urn:dgpe:names:draft:codice:schema:xsd:CommonBasicComponents-2',
    'cac-place-ext': 'urn:dgpe:names:draft:codice-place-ext:schema:xsd:CommonAggregateComponents-2',
    'cbc-place-ext': 'urn:dgpe:names:draft:codice-place-ext:schema:xsd:CommonBasicComponents-2',
}

# DataFrames base
entries_df = pd.DataFrame(columns=[
    'entry_id', 'link', 'summary', 'title', 'updated', 'contract_folder_id',
    'contract_folder_status', 'procurement_project_type_code', 'procurement_project_subtype_name',
    'budget_currency', 'estimated_overall_contract_amount', 'total_amount', 'tax_exclusive_amount'
])

parties_df = pd.DataFrame(columns=['nif'])
documents_df = pd.DataFrame(columns=[])

# Diccionario para traducir el estado
status_codes_dict = {
    "ADJ": "Adjudicado",
    "PUB": "Publicado",
    "EV": "En evaluación",
    "RES": "Resuelto",
    "PRE": "Anuncio Previo"
}

# Parseamos cada 'entry' dentro del XML principal
for entry in root.findall('atom:entry', ns):
    # Variables que iremos rellenando
    entry_id = link = title = summary = updated = None
    contract_folder_id = contract_folder_status_code = None
    pp_type_name = pp_subtype_name = pp_subentity_code = None
    pp_budget_currency_id = pp_estimated_overall_contract_amount = None
    pp_total_amount = pp_tax_exclusive_amount = None
    party_nif = None

    # Datos a nivel de entry
    entry_id_elem = entry.find('atom:id', ns)
    if entry_id_elem is not None:
        entry_id = entry_id_elem.text

    link_elem = entry.find('atom:link', ns)
    if link_elem is not None:
        link = link_elem.get('href')

    summary_elem = entry.find('atom:summary', ns)
    if summary_elem is not None:
        summary = summary_elem.text

    title_elem = entry.find('atom:title', ns)
    if title_elem is not None:
        title = title_elem.text

    updated_elem = entry.find('atom:updated', ns)
    if updated_elem is not None:
        updated = updated_elem.text

    write_row = True  # Para controlar si escribimos la fila final en entries_df

    # Elemento principal de Contrato
    cfs_elem = entry.find('cac-place-ext:ContractFolderStatus', ns)
    if cfs_elem is not None:
        # ID del expediente
        cfolder_id_elem = cfs_elem.find('cbc:ContractFolderID', ns)
        if cfolder_id_elem is not None:
            contract_folder_id = cfolder_id_elem.text

        # Estado del expediente (código)
        cfolder_status_code_elem = cfs_elem.find('cbc-place-ext:ContractFolderStatusCode', ns)
        if cfolder_status_code_elem is not None:
            contract_folder_status_code = cfolder_status_code_elem.text
            contract_folder_status = status_codes_dict.get(contract_folder_status_code, contract_folder_status_code)
        else:
            contract_folder_status = None

        # Control para no duplicar registros con la misma ContractFolderID si la que está ya en DF es más reciente
        if len(entries_df[entries_df['contract_folder_id'] == contract_folder_id].values) > 0:
            prev_updated = entries_df['updated'][entries_df['contract_folder_id'] == contract_folder_id].values
            if len(prev_updated) > 0:
                if prev_updated.max() > updated:
                    write_row = False

        if write_row:
            # ProcurementProject
            pp_elem = cfs_elem.find('cac:ProcurementProject', ns)
            if pp_elem is not None:
                # Tipo de contrato (por ejemplo: Obras, Suministros, etc.)
                pp_type_code_elem = pp_elem.find('cbc:TypeCode', ns)
                if pp_type_code_elem is not None:
                    pp_type_code = pp_type_code_elem.text
                    # Buscamos el nombre en df_contract_codes
                    mask_type = df_contract_codes['contract_code'] == pp_type_code
                    if any(mask_type):
                        pp_type_name = df_contract_codes.loc[mask_type, 'contract_name'].values[0]
                    else:
                        pp_type_name = None
                
                # Subtipo de contrato
                pp_subtype_code_elem = pp_elem.find('cbc:SubTypeCode', ns)
                if pp_subtype_code_elem is not None:
                    pp_subtype_code = pp_subtype_code_elem.text
                    subtype_listuri = pp_subtype_code_elem.get('listURI')
                    if subtype_listuri == 'http://contrataciondelestado.es/codice/cl/1.04/ServiceContractCode-1.04.gc':
                        mask_subtype = df_service_contract_codes['service_contract_code'] == pp_subtype_code
                        if any(mask_subtype):
                            pp_subtype_name = df_service_contract_codes.loc[mask_subtype, 'service_contract_name'].values[0]
                    elif subtype_listuri == 'http://contrataciondelestado.es/codice/cl/1.04/GoodsContractCode-1.04.gc':
                        mask_subtype = df_goods_contract_codes['goods_contract_code'] == pp_subtype_code
                        if any(mask_subtype):
                            pp_subtype_name = df_goods_contract_codes.loc[mask_subtype, 'goods_contract_name'].values[0]
                    elif subtype_listuri == 'http://contrataciondelestado.es/codice/cl/2.02/PatrimonialContractCode-2.02.gc':
                        mask_subtype = df_patrimonial_contract_codes['patrimonial_contract_code'] == pp_subtype_code
                        if any(mask_subtype):
                            pp_subtype_name = df_patrimonial_contract_codes.loc[mask_subtype, 'patrimonial_contract_name'].values[0]
                    else:
                        pp_subtype_name = None

                # Presupuesto
                pp_budget_amount = pp_elem.find('cac:BudgetAmount', ns)
                if pp_budget_amount is not None:
                    eoca_elem = pp_budget_amount.find('cbc:EstimatedOverallContractAmount', ns)
                    if eoca_elem is not None:
                        pp_budget_currency_id = eoca_elem.get('currencyID')
                        pp_estimated_overall_contract_amount = eoca_elem.text

                    total_amount_elem = pp_budget_amount.find('cbc:TotalAmount', ns)
                    if total_amount_elem is not None:
                        pp_total_amount = total_amount_elem.text

                    tax_excl_amount_elem = pp_budget_amount.find('cbc:TaxExclusiveAmount', ns)
                    if tax_excl_amount_elem is not None:
                        pp_tax_exclusive_amount = tax_excl_amount_elem.text

                # Localización
                pp_realized_location = pp_elem.find('cac:RealizedLocation', ns)
                if pp_realized_location is not None:
                    csc_elem = pp_realized_location.find('cbc:CountrySubentityCode', ns)
                    if csc_elem is not None:
                        pp_subentity_code = csc_elem.text

            # Contratista / entidad contratante
            lcp_elem = cfs_elem.find('cac-place-ext:LocatedContractingParty', ns)
            if lcp_elem is not None:
                party_elem = lcp_elem.find('cac:Party', ns)
                if party_elem is not None:
                    # Identificaciones
                    party_dir3 = None
                    party_id_plataforma = None
                    for party_identification_elem in party_elem.findall('cac:PartyIdentification', ns):
                        party_identification_id_elem = party_identification_elem.find('cbc:ID', ns)
                        if party_identification_id_elem is not None:
                            scheme_name = party_identification_id_elem.get('schemeName')
                            if scheme_name == 'DIR3':
                                party_dir3 = party_identification_id_elem.text
                            elif scheme_name == 'NIF':
                                party_nif = party_identification_id_elem.text
                            elif scheme_name == 'ID_PLATAFORMA':
                                party_id_plataforma = party_identification_id_elem.text

                    # Si no existe ya la entidad en parties_df
                    if party_nif not in parties_df['nif'].values:
                        # Nombre y otros datos
                        party_name_elem = party_elem.find('cac:PartyName', ns)
                        party_name = party_name_elem.find('cbc:Name', ns).text if party_name_elem is not None else None
                        website_uri_elem = party_elem.find('cbc:WebsiteURI', ns)
                        website_uri = website_uri_elem.text if website_uri_elem is not None else None

                        contracting_party_type_code_elem = lcp_elem.find('cbc:ContractingPartyTypeCode', ns)
                        contracting_party_type_code = contracting_party_type_code_elem.text if contracting_party_type_code_elem is not None else None

                        activity_code_elem = lcp_elem.find('cbc:ActivityCode', ns)
                        activity_code = activity_code_elem.text if activity_code_elem is not None else None

                        # Dirección
                        city_name = postal_zone = address_line = country_identification_code = None
                        postal_address_elem = party_elem.find('cac:PostalAddress', ns)
                        if postal_address_elem is not None:
                            city_name_elem = postal_address_elem.find('cbc:CityName', ns)
                            if city_name_elem is not None:
                                city_name = city_name_elem.text

                            postal_zone_elem = postal_address_elem.find('cbc:PostalZone', ns)
                            if postal_zone_elem is not None:
                                postal_zone = postal_zone_elem.text

                            address_line_elem = postal_address_elem.find('cac:AddressLine', ns)
                            if address_line_elem is not None:
                                line_elem = address_line_elem.find('cbc:Line', ns)
                                address_line = line_elem.text if line_elem is not None else None

                            country_elem = postal_address_elem.find('cac:Country', ns)
                            if country_elem is not None:
                                identification_code_elem = country_elem.find('cbc:IdentificationCode', ns)
                                if identification_code_elem is not None:
                                    country_identification_code = identification_code_elem.text

                        # Datos de contacto
                        telephone = telefax = electronic_mail = None
                        contact_elem = party_elem.find('cac:Contact', ns)
                        if contact_elem is not None:
                            telephone_elem = contact_elem.find('cbc:Telephone', ns)
                            if telephone_elem is not None:
                                telephone = telephone_elem.text

                            telefax_elem = contact_elem.find('cbc:Telefax', ns)
                            if telefax_elem is not None:
                                telefax = telefax_elem.text

                            electronic_mail_elem = contact_elem.find('cbc:ElectronicMail', ns)
                            if electronic_mail_elem is not None:
                                electronic_mail = electronic_mail_elem.text

                        # Añadimos fila a parties_df
                        new_party = pd.DataFrame([{
                            'dir3': party_dir3,
                            'nif': party_nif,
                            'id_plataforma': party_id_plataforma,
                            'name': party_name,
                            'website_uri': website_uri,
                            'type_code': contracting_party_type_code,
                            'activity_code': activity_code,
                            'party_subentity_code': pp_subentity_code,
                            'party_postal_zone': postal_zone,
                            'party_address_line': address_line,
                            'telephone': telephone,
                            'telefax': telefax,
                            'email': electronic_mail
                        }])
                        parties_df = pd.concat([parties_df, new_party], ignore_index=True)

            # Documentos
            # LegalDocumentReference (Pliego Administrativo)
            ldr_elem = cfs_elem.find('cac:LegalDocumentReference', ns)
            if ldr_elem is not None:
                legal_document_reference_id_elem = ldr_elem.find('cbc:ID', ns)
                if legal_document_reference_id_elem is not None:
                    legal_document_reference_id = legal_document_reference_id_elem.text
                else:
                    legal_document_reference_id = None
                
                attach_elem = ldr_elem.find('cac:Attachment', ns)
                if attach_elem is not None:
                    ext_ref_elem = attach_elem.find('cac:ExternalReference', ns)
                    if ext_ref_elem is not None:
                        uri_elem = ext_ref_elem.find('cbc:URI', ns)
                        if uri_elem is not None:
                            legal_document_uri = uri_elem.text
                            documents_df = pd.concat([
                                documents_df,
                                pd.DataFrame([{
                                    'contract_id': contract_folder_id,
                                    'document_reference_id': legal_document_reference_id,
                                    'document_uri': legal_document_uri,
                                    'document_type': 'Pliego Administrativo'
                                }])
                            ], ignore_index=True)

            # TechnicalDocumentReference (Pliego Técnico)
            tdr_elem = cfs_elem.find('cac:TechnicalDocumentReference', ns)
            if tdr_elem is not None:
                technical_document_reference_id_elem = tdr_elem.find('cbc:ID', ns)
                if technical_document_reference_id_elem is not None:
                    technical_document_reference_id = technical_document_reference_id_elem.text
                else:
                    technical_document_reference_id = None

                t_attach_elem = tdr_elem.find('cac:Attachment', ns)
                if t_attach_elem is not None:
                    t_ext_ref_elem = t_attach_elem.find('cac:ExternalReference', ns)
                    if t_ext_ref_elem is not None:
                        t_uri_elem = t_ext_ref_elem.find('cbc:URI', ns)
                        if t_uri_elem is not None:
                            technical_document_uri = t_uri_elem.text
                            documents_df = pd.concat([
                                documents_df,
                                pd.DataFrame([{
                                    'contract_id': contract_folder_id,
                                    'document_reference_id': technical_document_reference_id,
                                    'document_uri': technical_document_uri,
                                    'document_type': 'Pliego Técnico'
                                }])
                            ], ignore_index=True)

            # AdditionalDocumentReference (Documentos adicionales)
            additional_docs = cfs_elem.findall('cac:AdditionalDocumentReference', ns)
            if additional_docs:
                for adr_elem in additional_docs:
                    additional_document_reference_id_elem = adr_elem.find('cbc:ID', ns)
                    if additional_document_reference_id_elem is not None:
                        additional_document_reference_id = additional_document_reference_id_elem.text
                    else:
                        additional_document_reference_id = None

                    a_attach_elem = adr_elem.find('cac:Attachment', ns)
                    if a_attach_elem is not None:
                        a_ext_ref_elem = a_attach_elem.find('cac:ExternalReference', ns)
                        if a_ext_ref_elem is not None:
                            a_uri_elem = a_ext_ref_elem.find('cbc:URI', ns)
                            if a_uri_elem is not None:
                                additional_document_uri = a_uri_elem.text
                                documents_df = pd.concat([
                                    documents_df,
                                    pd.DataFrame([{
                                        'contract_id': contract_folder_id,
                                        'document_reference_id': additional_document_reference_id,
                                        'document_uri': additional_document_uri,
                                        'document_type': 'Documento adicional'
                                    }])
                                ], ignore_index=True)

            # Finalmente, añadimos la fila a entries_df
            new_entry = pd.DataFrame([{
                'entry_id': entry_id,
                'link': link,
                'summary': summary,
                'title': title,
                'updated': updated,
                'contract_folder_id': contract_folder_id,
                'contract_folder_status': contract_folder_status,
                'procurement_project_type_code': pp_type_name,
                'procurement_project_subtype_name': pp_subtype_name,
                'budget_currency': pp_budget_currency_id,
                'estimated_overall_contract_amount': pp_estimated_overall_contract_amount,
                'total_amount': pp_total_amount,
                'tax_exclusive_amount': pp_tax_exclusive_amount,
                'party_nif': party_nif
            }])
            entries_df = pd.concat([entries_df, new_entry], ignore_index=True)

print("Número de licitaciones parseadas:", len(entries_df))

## 5. Creación de la Base de Datos PostgreSQL

En esta sección se establecen las credenciales de la base de datos y se crea el motor de conexión con SQLAlchemy.
Se asume que ya se tiene corriendo una instancia de PostgreSQL y las credenciales son correctas.

**Nota**: Ajusta usuario, contraseña y nombre de base de datos según tus necesidades.


In [ ]:
from sqlalchemy import create_engine
from urllib.parse import quote
from sqlalchemy.engine import URL

# Credenciales
usuario = 'postgres'
password = 'place_rag_password'
host = 'localhost'  # Ajustar si no está en localhost
puerto = '5432'     # Puerto por defecto
base_datos = 'place_rag_db'

# Codificar credenciales
password_cifrada = quote(password)
usuario_cifrado = quote(usuario)
host_cifrado = quote(host)

# Crear la URL de conexión con la contraseña codificada
url = URL.create(
    drivername="postgresql+psycopg2",
    username=usuario_cifrado,
    password=password_cifrada,
    host=host_cifrado,
    port=puerto,
    database=base_datos,
)

# Crear el motor de conexión
engine = create_engine(
    url,
    connect_args={"options": "-c client_encoding=UTF8"}
)
print("Motor de conexión a PostgreSQL creado con éxito.")

## 6. Definición del esquema y creación de tablas

A continuación, usando `SQLAlchemy`, se definen las tablas necesarias:
1. `paises`
2. `regiones`
3. `entidades`
4. `expedientes`
5. `documentos`

Cada tabla incluye las columnas, tipos de datos y relaciones (Foreign Keys) oportunas. Posteriormente, `metadata.create_all(engine)` crea las tablas en la base de datos si no existen.


In [ ]:
from sqlalchemy import MetaData, Table, Column, Numeric, String, ForeignKey

# Objeto MetaData para agrupar la definición del esquema
metadata = MetaData()

# Definir la tabla paises
paises_table = Table(
    "paises",
    metadata,
    Column("country_code", String, primary_key=True),
    Column("country_name", String),
)

# Definir la tabla regiones
regiones_table = Table(
    "regiones",
    metadata,
    Column("country_subentity_code", String, primary_key=True),
    Column("country_subentity_name", String),
    Column("country_code", String, ForeignKey("paises.country_code")),
)

# Definir la tabla entidades
entidades_table = Table(
    "entidades",
    metadata,
    Column("nif", String, primary_key=True),
    Column("name", String),
    Column("website_uri", String),
    Column("type_code", String),
    Column("activity_code", String),
    Column("party_subentity_code", String, ForeignKey("regiones.country_subentity_code")),
    Column("party_postal_zone", String),
    Column("party_address_line", String),
    Column("telephone", String),
    Column("telefax", String),
    Column("email", String),
    Column("dir3", String),
    Column("id_plataforma", String),
)

# Definir la tabla expedientes
expedientes_table = Table(
    "expedientes",
    metadata,
    Column("contract_folder_id", String, primary_key=True),
    Column("entry_id", String),
    Column("link", String),
    Column("summary", String),
    Column("title", String),
    Column("updated", String),
    Column("contract_folder_status", String),
    Column("procurement_project_type_code", String),
    Column("procurement_project_subtype_name", String),
    Column("budget_currency", String),
    Column("estimated_overall_contract_amount", Numeric(12, 2)),
    Column("total_amount", Numeric(12, 2)),
    Column("tax_exclusive_amount", Numeric(12, 2)),
    Column("party_nif", String, ForeignKey("entidades.nif")),
)

# Definir la tabla documentos
documentos_table = Table(
    "documentos",
    metadata,
    Column("document_reference_id", String),
    Column("document_uri", String, primary_key=True),
    Column("document_type", String),
    Column("contract_id", String, ForeignKey("expedientes.contract_folder_id"))
)

# Crear las tablas en la base de datos
metadata.create_all(engine)
print("Tablas creadas (si no existían).")

## 7. Inserción de los datos en la base de datos
A continuación, se muestra cómo insertar los datos de los DataFrames en las tablas creadas usando el método `.to_sql(...)` de `pandas`.

> **Nota**: Dependiendo de la cantidad de datos y restricciones, puede que desees utilizar un método de inserción por lotes más optimizado o manejar potenciales conflictos (duplicados, etc.) con mayor detalle. Aquí hacemos un simple `append`.


In [ ]:
print("Insertando DataFrames en tablas...")

# Insertamos los países
df_country_codes.to_sql("paises", engine, if_exists="append", index=False)

# Insertamos las regiones (subentidades)
df_country_subentity_codes.to_sql("regiones", engine, if_exists="append", index=False)

# Insertamos las entidades
parties_df.to_sql("entidades", engine, if_exists="append", index=False)

# Insertamos los expedientes
entries_df.to_sql("expedientes", engine, if_exists="append", index=False)

# Insertamos los documentos
documents_df.to_sql("documentos", engine, if_exists="append", index=False)

print("Datos insertados correctamente en PostgreSQL.")

## 8. Conclusiones

En este notebook hemos:
- Parseado un archivo XML con licitaciones públicas.
- Descargado y creado DataFrames de tablas de códigos (tipos de contratos, países, etc.).
- Creado la estructura de una base de datos PostgreSQL con `SQLAlchemy`.
- Insertado los datos (expedientes, documentos, entidades, etc.) en las tablas.

A partir de aquí, se pueden realizar consultas SQL en la base de datos para explotar la información de licitaciones.

¡Fin del proceso!